# 해당 파일(pre_analysis.ipynb)의 목적

본격적인 분석을 진행하기 전에 오류(반대로 된 포켓몬 순위)를 수정하고 신뢰성에 관한 간단한 검증을 수행합니다.

In [1]:
# 필요한 패키지를 불러옵니다.

import pandas as pd
import numpy as np

In [2]:
# 처음 받았던 파일과 승률 데이터 파일을 불러옵니다.

pokemon = pd.read_csv("C:\\nexon\\TypeA\\pokemon.csv", encoding = "utf-8")
winner = pd.read_csv("pokemon_winner_rate.csv", encoding = "utf-8")

In [3]:
# feature를 확인합니다

pokemon.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

In [4]:
# 앞 과정(stage.ipynb) 에서 잘못 구한 Rank를 수정합니다.

winner["TotalRank"] = winner["CountOf_p90"].rank(ascending = False)

In [5]:
# 앞 과정(stage.ipynb) 에서 잘못 구한 Rank를 수정합니다.

types = pokemon.loc[:, "type1"].unique()
winner["TypeRank"] = np.nan

for t in types:
    typerank = winner[pokemon["type1"] == t]["TotalRank"].rank()
    winner["TypeRank"].iloc[typerank.keys()] = typerank.values

C:\Users\pqrs\Anaconda3\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
# 처음 전달받은 파일에 MeanOf_p??, CountOf_p??, 포켓몬 전체 순위와 타입 내 순위, BMI를 삽입합니다.
# BMI는 일반적으로 사람의 비만도를 측정하는 무게(kg)/키(m)^2 을 의미합니다.

for i in range(10):
    pokemon["MeanOf_p" + str(i) + "0"] = winner["MeanOf_p" + str(i) + "0"]

for i in range(10):
    pokemon["CountOf_p" + str(i) + "0"] = winner["CountOf_p" + str(i) + "0"]

pokemon["TotalRank"] = winner["TotalRank"]
pokemon["TypeRank"] = winner["TypeRank"]
pokemon["BMI"] = pokemon["weight_kg"] / (pokemon["height_m"]**2)

In [7]:
# 정보를 추가한 파일을 저장합니다.

pokemon.to_csv("pokemon_chart.csv", encoding = "utf-8", index = False)

In [8]:
# 분석에 앞서 믿을만한 시뮬레이션 결과를 얻었는지 파악해보고자 같은 포켓몬끼리 싸운 배틀의 승률의 평균을 구합니다.
# 동전의 앞면과 뒷면이 나올 확률은 1/2로 동일하지만, 동전을 10번 던진다고 5번 앞면, 5번 뒷면이 나오지 않는 것과 같은 이치입니다.
# 같은 포켓몬끼리 싸운 경기의 승률은 이론적으로 0.50입니다.

s = 0
for i in range(801):
    s += winner.iloc[i, i+1]
s = s/801.0

In [9]:
# 본인과 본인이 싸운 경기(포켓몬 801종류 * 100회)를 모두 평균내본 결과 0.50166... 이란 결과를 얻었습니다.

s

0.50166042446941272

In [10]:
# 이항분포 B(n, 1/2) / n ~ N(1/2, (1/2*sqrt(n))^2) 이므로 이를 활용해 검증해보자 합니다.
# 계산 결과 약 Z = 0.94 로 나타났습니다.
# -1.96 < Z < 1.96 이 95% 신뢰구간인 것을 생각했을 때
# Z = 0.94란 결과는 안정적으로 신뢰구간에 안착해 있는 것을 알 수 있습니다.

(s - 0.5)/(0.5/((801*100)**(1/2.)))

0.93986478693868436